# Data Augmentation Box

Project for Data Augmentation System

## Data Augmentation Order

STEP 1 - Domain Data Preparation
1. Domain data labeling check
2. Dimensionality Reduction
3. Regression analysis


STEP 2 - Data Augmentation
1. Domain data check
02. Crawling
03. Data filtering (1st)
04. Dimensionality Reduction
05. Label Spreading (semi-supervised learning based)
06. Data Filtering (2nd)
07. Regression analytsis
08. Data Filtering (3rd)
09. Data Augmentation
10. Model Generation

- - -

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 1. Domain Data Check

we have to check the domain data 

### 1-1) image dataset

In [ ]:
domain =

### 1-2) numerical dataset

In [ ]:
### HRV numerical dataset
domain = pd.read_csv('E:/RESEARCH/Datasets/HRV/HRV_REV_all.csv', sep=',')

In [ ]:
### data shape check
domain.shape

In [ ]:
### data variable columns check
print(domain.columns)
domain.head()

In [ ]:
### checking lables for the data
domain.loc[domain['HAMD']<=7, 'IndexH'] =0
domain.loc[(domain['HAMD']>7) & (domain['HAMD']<=16), 'IndexH'] = 1
domain.loc[domain['HAMD']>16, 'IndexH'] = 2
domain_y = domain.loc[:,'IndexH']
# domain_y = domain.loc[:,'disorder']

In [ ]:
### deleting unnecessary data columns
domain = domain.drop(['sub','age','gender','VISIT','disorder','HAMD', 'HAMA','PDSS','ASI','APPQ','PSWQ','SPI','PSS','BIS','SSI','IndexH'], axis=1)

In [ ]:
### check the domain data columns again
print(domain.columns)
print(domain.shape)

## Python data crawling

we proceed data crawling to support insufficient data environment

In [ ]:
import urllib.request
import time
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
from icrawler.builtin import GoogleImageCrawler

In [ ]:
# google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4,
#                                     storage={'root_dir': 'E:/RESEARCH/Datasets/HRV/crawl_test'})

# google_crawler.crawl(keyword='car crash', max_num=500,
# #                      date_min=None, date_max=None,
#                      min_size=(200,200), max_size=None)

In [ ]:
def scroll_down():
    global driver
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            time.sleep(SCROLL_PAUSE_SEC)
            new_height = driver.execute_script("return document.body.scrollHeight")

            try:
                driver.find_element_by_class_name("mye4qd").click()
            except:

               if new_height == last_height:
                   break


        last_height = new_height

In [ ]:
keyword = input('enter your keyword : ')
url = 'https://www.google.com/search?q={}&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjgwPKzqtXuAhWW62EKHRjtBvcQ_AUoAXoECBEQAw&biw=768&bih=712'.format(keyword)

driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)

scroll_down()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
images = soup.find_all('img', attrs={'class':'rg_i Q4LuWd'})

print('number of img tags: ', len(images))

n = 1
for i in images:

    try:
        imgUrl = i["src"]
    except:
        imgUrl = i["data-src"]
        
    with urllib.request.urlopen(imgUrl) as f:
        with open('./img/' + keyword + str(n) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)

    n += 1

In [1]:
from icrawler.builtin import GoogleImageCrawler

# Google에서 크롤링
google_crawler = GoogleImageCrawler(
    feeder_threads=1,
    parser_threads=1,
    downloader_threads=4,
    storage={'root_dir': 'E:/RESEARCH/Datasets/HRV/crawl_test'})
filters = dict(
    size='large',
    color='orange',
    license='commercial,modify',
    date=((2017, 1, 1), (2017, 11, 30)))
google_crawler.crawl(keyword='car', filters=filters, offset=0, max_num=1000,
                     min_size=(200,200), max_size=None, file_idx_offset=0)

2022-03-11 15:04:48,643 - INFO - icrawler.crawler - start crawling...
2022-03-11 15:04:48,645 - INFO - icrawler.crawler - starting 1 feeder threads...
2022-03-11 15:04:48,646 - INFO - icrawler.crawler - starting 1 parser threads...
2022-03-11 15:04:48,646 - INFO - icrawler.crawler - starting 4 downloader threads...
2022-03-11 15:04:49,861 - INFO - parser - parsing result page https://www.google.com/search?q=car&ijn=0&start=0&tbs=isz%3Al%2Cic%3Aspecific%2Cisc%3Aorange%2Csur%3Afmc%2Ccdr%3A1%2Ccd_min%3A01%2F01%2F2017%2Ccd_max%3A11%2F30%2F2017&tbm=isch
2022-03-11 15:04:51,472 - INFO - downloader - image #1	https://get.pxhere.com/photo/car-wheel-vehicle-sports-car-supercar-lamborghini-land-vehicle-automobile-make-automotive-design-luxury-vehicle-performance-car-lamborghini-aventador-185864.jpg
2022-03-11 15:04:51,580 - INFO - downloader - image #2	https://get.pxhere.com/photo/mc-Laren-pkw-land-vehicle-vehicle-car-supercar-sports-car-automotive-design-orange-mclaren-automotive-Mclaren-p1-per